# Instalacion de dependencias

In [1]:
#pip install happytransformer

In [22]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim #Pytorch
from datasets import load_dataset

In [23]:
RANDOM_SEED = 42
BATCH_SIZE = 16
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Cargar datos

In [24]:
dataset = load_dataset('csv', data_files=['C:/Users/anabe/Desktop/PRUEBAS/APPSTORE1 - ACTUALIZADO/corpus/book4.csv'], split='train')
datasets = dataset.train_test_split(test_size=0.20) # Test del 10%

Found cached dataset csv (C:/Users/anabe/.cache/huggingface/datasets/csv/default-093a8e447c963488/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Loading cached split indices for dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-d995bbfd5f578a3e.arrow and C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-974f951674d0b4c2.arrow


In [25]:
datasets

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip'],
        num_rows: 625
    })
})

In [5]:
def etiquetas(examples):
    if examples["general"] == 'Linguistics':
        label = 0
    else: 
        label = 1
    return {"labels": label}

In [26]:
def etiquetas(examples):
    if examples ["general"] == 'applied science':
        label = 0
    elif examples ["general"] == 'natural sciences':
        label = 1
    elif examples ["general"] == 'math':
        label = 2
    elif examples ["general"] == 'social sciences ':
        label = 3
    elif examples ["general"] == 'art':
        label = 4
    elif examples ["general"] == 'linguistics':
        label = 5
    else:
        label = 6
    return {"labels": label}

In [27]:
dataset = datasets.map(etiquetas)

Loading cached processed dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-8805b999bb461f46.arrow
Loading cached processed dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-0d6d123db84ac67f.arrow


In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip', 'labels'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip', 'labels'],
        num_rows: 625
    })
})

In [29]:
dataset['train'][5]

{'name': 'Ananas Language Exchange',
 'general': 'linguistics',
 'descrip': 'The language app for today. Learn and exchange languages. Think global, be local.',
 'labels': 5}

## Tokenizacion DataSet

In [30]:
model_checkpoint = "distilbert-base-uncased"

In [31]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#, use_fast=False

In [32]:
#Vamos a pasar el tokenizador en cada fila del dataset
def tokenize_reviews(examples):
    return tokenizer(examples["descrip"], padding=True, truncation=True)

In [33]:
type(dataset['train']['descrip'][0])

str

In [34]:
encoded_dataset = dataset.map(tokenize_reviews, batched=True )

Loading cached processed dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-9ae0891174e85f96.arrow


Map:   0%|          | 0/625 [00:00<?, ? examples/s]

In [35]:
tokenizer.vocab_size

30522

## Cargar el modelo preentrenado

In [36]:
from transformers import AutoModelForSequenceClassification

num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [37]:
encoded_dataset['train'].features
#encoded_dataset

{'name': Value(dtype='string', id=None),
 'general': Value(dtype='string', id=None),
 'descrip': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [38]:
encoded_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])
encoded_dataset['train'].features

{'name': Value(dtype='string', id=None),
 'general': Value(dtype='string', id=None),
 'descrip': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [39]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 625
    })
})

In [40]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [41]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 64
num_train_epochs=4
num_train_samples = 2_000
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
#train_dataset = encoded_dataset["train"]

#logging_steps = len(encoded_dataset['train'])//batch_size
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="25sep",
    num_train_epochs=num_train_epochs,     
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch", 
    logging_steps=logging_steps,
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-wiki_multi"
)

In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

In [43]:
trainer.train()

C:\Users\anabe\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.339400,1.272787,0.601600,0.480655
2,0.883700,0.948998,0.716800,0.662473
3,0.829100,0.828810,0.752000,0.704604
4,0.747400,0.795387,0.758400,0.711019


TrainOutput(global_step=128, training_loss=1.047879045829177, metrics={'train_runtime': 9204.7951, 'train_samples_per_second': 0.869, 'train_steps_per_second': 0.014, 'total_flos': 585806508912000.0, 'train_loss': 1.047879045829177, 'epoch': 4.0})